In [1]:
# %%
from transformers import pipeline, set_seed
from datasets import load_dataset
import pandas as pd
import json
import regex as re
import nltk
from nltk.tokenize import sent_tokenize
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import ast
import torch
from huggingface_hub import scan_cache_dir
import numpy as np
from transformers import AutoTokenizer, AutoModel, AutoConfig
import ast
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
import os
from datasets import DatasetDict, concatenate_datasets
from sentence_transformers import SentenceTransformer
from joblib import dump, load
import argparse
import joblib

#os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

#watch -n0.1 nvidia-smi

/Users/oluwaseuncardoso/Desktop/Projects/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Load model

cuda_count = torch.cuda.device_count()

sentence_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2').to(device)

# model_name  = "Open-Orca/Mistral-7B-OpenOrca" #"mistralai/Mistral-7B-v0.1
# config = AutoConfig.from_pretrained(model_name)
# max_input_size =  config.max_position_embeddings  
# tokenizer = AutoTokenizer.from_pretrained(model_name, device=device, )


In [2]:
def getIdsType(brief_type):
    def getIds(briefs):

        briefs = briefs[brief_type]
        briefs_ids = []
        briefs = ast.literal_eval(briefs)
        for brief in briefs:
            iD = tokenizer(brief, max_length = max_input_size, padding='max_length', truncation= True ,return_tensors="pt") #.to(device).input_ids 
            briefs_ids.append(iD)
        return { f"ids_{brief_type}": briefs_ids }
    return getIds


def meanEmbeddings(briefs):

    briefs = ast.literal_eval(briefs)
    # Place model inputs on the GPU
    embeddings = []
    for brief in briefs:
        argument = tokenizer(brief, max_length = max_input_size , padding="max_length" ,truncation= True ,return_tensors="pt").to(device) 
        # Extract last hidden states
        model.eval()
        with torch.no_grad():
            argument = argument.to(device)
            output = model(**argument)
            argument = argument.to(device)
            last_hidden_state = output.last_hidden_state


        last_hidden_state = last_hidden_state.reshape(( max_input_size, config.hidden_size))
        mask = argument['attention_mask'].to(device).bool()
        mask = mask.reshape(max_input_size)
        mask = mask.nonzero().squeeze()
        hidden_states = torch.index_select(last_hidden_state, 0, mask)
        
        #print(last_hidden_state)
        inputs = get_mean_embedding(hidden_states.cpu().to(torch.float64).numpy()).tolist()
        del last_hidden_state
        del output
        del mask
        del hidden_states
        del argument
        
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
                
        embeddings.append(inputs)
    # Return vector for [CLS] token
    return embeddings

def summerizeEmbeddings(brief):
    embedding = sentence_model.encode(brief).tolist()
    return embedding



def get_mean_embedding(embedding):
    return np.mean(embedding, axis=0)


In [3]:
model_name = "KNN"
feature = 'tfidf'
#for model_name in ["RFT", "SGD"]: #["KNN","LinearSVC", "Logistic", "RFT", "SGD"]:
data = 'dataset/testset.csv'
if True: #__name__ == "__main__":
    model_types = ["KNN","LinearSVC", "Logistic", "RFT", "SGD"]
    # parser = argparse.ArgumentParser(description='End to end pipeline from briefs to predictions')
    # parser.add_argument('--model_name', default='RFT', help=f'There are {len(model_types)} models to choose from: {model_types}')
    # parser.add_argument('--data',  default= 'embeddings.csv')
    # parser.add_argument('--feature', default='sentence_embeddings', help='There are two features to choose from: sentence_embeddings and tfidf')



    # arg = parser.parse_args()

    testset = pd.read_csv(data, index_col=0)#pd.read_csv(arg.data, index_col=0)

    testset["completion"] = list(map(lambda x : x.strip() ,testset["completion"].to_list()))
    

    summerize = True

    if feature == 'sentence_embeddings':#if arg.feature == 'sentence_embeddings':
        if 'embeddings' not in testset.columns:
            if not summerize:
                model  = "mistralai/Mistral-7B-v0.1"
                config = AutoConfig.from_pretrained(model)
                max_input_size =  config.max_position_embeddings  
                tokenizer = AutoTokenizer.from_pretrained(model, device=device, )

                tokenizer.pad_token = tokenizer.eos_token
                model = AutoModel.from_pretrained(model, torch_dtype= torch.bfloat16 ).to(device)  #,device_map = "auto"
                #parallel_model = torch.nn.DataParallel(model)
            else:
                sentence_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2').to(device)

            testset['feature'] = ""
            testset['feature'] = testset['prompt'].map(summerizeEmbeddings)
            testset.to_csv('embeddings.csv')


        else:
            #get model for predictions
            turn2list = lambda x: ast.literal_eval(x)
            testset['feature'] = testset['embeddings'].map(turn2list)


    elif feature == 'tfidf':# elif arg.feature == 'tfidf':
        print("Loading tfidf pipes")
        support_pipe = joblib.load('pipes/support-tfidf.joblib')
        opposition_pipe = joblib.load('pipes/oppose-tfidf.joblib')

        testset['feature'] = ""

        sparse_matrix = support_pipe.transform(testset["prompt"].loc[testset["brief_type"]=="support"]).toarray().tolist()

        testset.loc[testset["brief_type"]=="support" , 'feature'] = pd.Series( sparse_matrix , index = testset.loc[testset["brief_type"]=="support"].index)

        sparse_matrix = opposition_pipe.transform(testset["prompt"].loc[testset["brief_type"]=="opposition"]).toarray().tolist()

        testset.loc[testset["brief_type"]=="opposition", 'feature'] =  pd.Series( sparse_matrix , index = testset.loc[testset["brief_type"]=="opposition"].index)
        


    x_support = np.array(testset["feature"].loc[(testset["brief_type"]=="support") & (testset["data_type"]=="test") ].to_list())  #  np.array(testset['file_path'].to_list())
    x_opposition = np.array(testset["feature"].loc[(testset["brief_type"]=="opposition") & (testset["data_type"]=="test") ].to_list())  #  np.array(testset['file_path'].to_list())
        

    #get model for predictions

    support_model_path = f'models/{model_name}-support-{feature}.pkl'
    opposition_model_path = f'models/{model_name}-opposition-{feature}.pkl'

    clfs = {"sup" : load(support_model_path)  , "opp" : load(opposition_model_path) }

    if hasattr(clfs["sup"], 'predict_proba') and callable(getattr(clfs["sup"], 'predict_proba')):
        scores_support = clfs['sup'].predict_proba(x_support)
        scores_opposition = clfs['opp'].predict_proba(x_opposition)
    else:
        scores_support = clfs['sup'].decision_function(x_support)
        scores_opposition = clfs['opp'].decision_function(x_opposition)

    prediction_opposition = clfs['opp'].predict(x_opposition)
    prediction_support = clfs['sup'].predict(x_support)

    testset.rename(columns={"file_name": "brief","completion": "truth"} , inplace = True  )

    support = testset.loc[(testset["brief_type"]=="support") & (testset["data_type"]=="test") ].copy()
    opposition = testset.loc[(testset["brief_type"]=="opposition") & (testset["data_type"]=="test") ].copy()

    support.drop(['data_type','prompt' , 'brief_type','file_path','feature'], axis=1, inplace=True)
    opposition.drop(['data_type', 'prompt' , 'brief_type','file_path','feature'], axis=1, inplace=True)

    support['predict'] = ""
    opposition['predict'] = ""

    support['predict'] = prediction_support
    opposition['predict'] = prediction_opposition

    support['score'] = ""
    opposition['score'] = ""

    support['score'] = list(map( np.max ,scores_support.tolist()))
    opposition['score']= list(map(  np.max ,scores_opposition.tolist() ))

    support = support[["brief","predict","score","truth"]]
    opposition = opposition[["brief","predict","score","truth"]]

    # support.to_csv(f'predictions/{model_name}-{feature}-supppredictions.csv' , index = False)
    # opposition.to_csv(f'predictions/{model_name}-{feature}-oppopredictions.csv', index = False)


Loading tfidf pipes


In [13]:
n = 1
clfs['sup'].predict_proba(x_support[n].reshape(1,-1)), clfs['sup'].predict(x_support[n].reshape(1,-1))

(array([[0.4, 0.6]]), array(['grant'], dtype='<U163422'))

In [14]:
n = 0
clfs['sup'].predict_proba(x_support[n].reshape(1,-1)), clfs['sup'].predict(x_support[n].reshape(1,-1))

(array([[0.8, 0.2]]), array(['deny'], dtype='<U163422'))

In [108]:
testset['feature'] = ""

sparse_matrix = support_pipe.transform(testset["prompt"].loc[testset["brief_type"]=="support"]).toarray().tolist()

testset.loc[testset["brief_type"]=="support" , 'feature'] = pd.Series( sparse_matrix , index = testset.loc[testset["brief_type"]=="support"].index)

sparse_matrix = opposition_pipe.transform(testset["prompt"].loc[testset["brief_type"]=="opposition"]).toarray().tolist()

testset.loc[testset["brief_type"]=="opposition", 'feature'] =  pd.Series( sparse_matrix , index = testset.loc[testset["brief_type"]=="opposition"].index)



In [102]:
testset.loc[testset["brief_type"]=="opposition", 'feature'] 

1       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3       [0.015048255923431753, 0.0, 0.0, 0.0, 0.0, 0.0...
5       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
7       [0.0, 0.007986306346476821, 0.0, 0.0, 0.0, 0.0...
9       [0.011928884238130606, 0.0, 0.0, 0.0, 0.0, 0.0...
                              ...                        
1206    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1208    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1210    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1212    [0.003816286650750082, 0.06086658151642782, 0....
1214    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: feature, Length: 585, dtype: object

In [105]:
testset.loc[testset["brief_type"]=="opposition", 'feature'] 

1       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3       [0.015048255923431753, 0.0, 0.0, 0.0, 0.0, 0.0...
5       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
7       [0.0, 0.007986306346476821, 0.0, 0.0, 0.0, 0.0...
9       [0.011928884238130606, 0.0, 0.0, 0.0, 0.0, 0.0...
                              ...                        
1206    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1208    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1210    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1212    [0.003816286650750082, 0.06086658151642782, 0....
1214    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: feature, Length: 585, dtype: object

In [94]:
stuff = pd.Series( sparse_matrix.tolist() )
stuff

0      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1      [0.015048255923431753, 0.0, 0.0, 0.0, 0.0, 0.0...
2      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3      [0.0, 0.007986306346476821, 0.0, 0.0, 0.0, 0.0...
4      [0.011928884238130606, 0.0, 0.0, 0.0, 0.0, 0.0...
                             ...                        
580    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
581    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
582    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
583    [0.003816286650750082, 0.06086658151642782, 0....
584    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Length: 585, dtype: object

In [104]:
testset['feature'] = ""
sparse_matrix = opposition_pipe.transform(testset["prompt"].loc[testset["brief_type"]=="opposition"]).toarray().tolist()

testset.loc[testset["brief_type"]=="opposition", 'feature'] =  pd.Series( sparse_matrix , index= testset.loc[testset["brief_type"]=="opposition"].index) 

In [59]:
type(testset.loc[testset["brief_type"]=="support", 'feature']) ,  type(testset['feature'].loc[testset["brief_type"]=="support"])


(pandas.core.series.Series, pandas.core.series.Series)

In [66]:
testset['feature'].loc[testset["brief_type"]=="support"] = sparse_matrix.tolist()

/var/folders/jd/_y9k6hnx4658pxlgs6r0d1pr0000gn/T/ipykernel_62893/585839160.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testset['feature'].loc[testset["brief_type"]=="support"] = sparse_matrix.tolist()


In [73]:
testset.loc[testset["brief_type"]=="support", 'feature']

0       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4       [0.014406859280560716, 0.008113810186061055, 0...
6       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
8       [0.009605672960387038, 0.005409826360636613, 0...
                              ...                        
1205                                                  NaN
1207                                                  NaN
1209                                                  NaN
1211                                                  NaN
1213                                                  NaN
Name: feature, Length: 630, dtype: object

In [70]:
stuff = pd.Series([[i,i+9] for i in range(630)] ) # {"stuff" :[[i,i+9] for i in range(630)]}

testset.loc[testset["brief_type"]=="support", 'feature'] = stuff

In [84]:
stuff = [i for i in range(630)] #len(sparse_matrix.tolist())

In [83]:

testset.loc[testset["brief_type"]=="support", 'feature'] = pd.Series( sparse_matrix.tolist() )

In [ ]:
# 1. run experiment with the last hidden state with just support
# TFI-DF and Mistral7 embeddings, the claim is there isn't any statistical difference between the two
# hypothesis,

    # is having more information better than less information in classification
#hypothesis 1:
    # having more information is better than less information
#hypotheisis 2:
    # are TFI-DF embeddings better than Mistral7 embeddings
# After running experiment s